In [1]:
import numpy as np
import tensorflow as tf
import sentencepiece as spm
import re

### Parameter

In [2]:
document_tokenizer = spm.SentencePieceProcessor(model_file='document_spm.model')
summary_tokenizer = spm.SentencePieceProcessor(model_file='summary_spm.model')
summary_tokenizer.SetEncodeExtraOptions('bos:eos')

True

In [3]:
encoder_vocab_size = 400000
decoder_vocab_size = 100000

# vocab_size
encoder_vocab_size, decoder_vocab_size

(400000, 100000)

In [4]:
# maxlen
# taking values > and round figured to 75th percentile
# at the same time not leaving high variance
encoder_maxlen = 400
decoder_maxlen = 75

In [5]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [6]:
# hyper-params
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
EPOCHS = 12

### model

In [7]:
def cleaned_text(text):
    a = re.sub(r'\[', '&& [',text)
    a = re.sub(r'\]', '] &&',a)
    a = re.sub(r'&&', '', a )
    
    # (~~)의 양 옆에 띄어쓰기를 추가
    a = re.sub(r'\(', '&& (',a)
    a = re.sub(r'\)', ') &&',a)
    a = re.sub(r'&&', '', a )
    
    # <~~>의 양옆에 띄어쓰기를 추가
    a = re.sub(r'\<', '&& <',a)
    a = re.sub(r'\>', '> &&',a)
    a = re.sub(r'&&', '', a )
    
    # (~~)부분 삭제
    a = re.sub(r'\([^)]*\)', '', a )
    
    # [~~]부분 삭제
    a = re.sub(r'\[[^)]*\]', '', a )
    
    # <~~>부분 삭제
    
    a = re.sub(r'\<[^)]*\>', '', a )
    
    # 해당 패턴에 해당하는 부분 삭제
    pattern = '[\r|\n|\n\n]'
    a = re.sub(pattern = pattern, repl= ' ', string = a)
    
    # 이메일 삭제
    a = re.sub('[a-zA-Z0-9_-]+@\S+.+[a-zA-Z0-9]', '', a)
    #a = re.sub('[a-zA-Z0-9_-]+@[a-z]+.+[a-z]+.+[a-z]', '', a)
    #a = re.sub('[a-zA-Z0-9_-]+@[a-z]+.[a-z]', '', a)
    #a = re.sub('[a-z]+.[a-z]', '', a)
    
    # 날짜 삭제
    # 필요시 만들기
    
    
    # 마침표 뒤에 띄어쓰기
    #a = re.sub('\.', '. ', a) #숫자의 소수점으로 쓰이는 경우도 있기 때문에 토크나이저에게 맡김 
    
    # 두칸 이상의 띄어쓰기 제거
    a = re.sub('  ', ' ', a)
    a = re.sub('  ', ' ', a)
    a = re.sub('  ', ' ', a)
    
    # 양 옆에 공백제거 
    a = a.strip()
    
    return a

In [8]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [9]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


In [10]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [11]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

In [12]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

In [13]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights

In [14]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

In [15]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


In [16]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


In [17]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x


In [18]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights


In [19]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


In [20]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


In [21]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [22]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
)

In [23]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask


In [24]:
def evaluate(input_document):
    input_document = cleaned_text(input_document)
    input_document = document_tokenizer.EncodeAsIds(input_document)
    input_document = tf.keras.preprocessing.sequence.pad_sequences([input_document], maxlen=encoder_maxlen, padding='post', truncating='post')
    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [1] #[BOS] index
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == 2: #[EOS] index
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


In [25]:
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized, 0)
    return summary_tokenizer.DecodeIds(summarized[0].tolist())  # since there is just one translated document

#### Checkpoints

In [26]:
ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)
checkpoint_path = "checkpoints"
latest = tf.train.latest_checkpoint(checkpoint_path)
ckpt.restore(latest)

#### Test

10월 1일
전처리 O
카테고리 100 / 101 / 102 / 104 / 105
9월달 30일치

# 100(정치)

In [27]:
# https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=022&aid=0003508685
print("이재명 '징용판결 '정치개입' 요구한 스가, 방한할 일 없을 것'")
summarize('여권 차기 대선주자군에서 더불어민주당 이낙연 대표와 함께 ‘양강’으로 꼽히는 이재명 경기지사가 1일 스가 요시히데 일본 총리가 한국을 방문하는 일은 없을 것이라고 단언했다. 민족의 명절인 추석날에 일본 총리에 대한 실망감을 드러내며 ‘반일 이미지’를 부각한 셈이다. 최근 여론조사에서 이 대표 지지율이 근소한 차로 1위를 되찾으며 상승세를 탄 것과 무관치 않다는 분석이 나온다. 이 지사는 이날 자신의 페이스북에 ‘스가총리가 방한할 일은 없겠습니다’라는 제목의 글을 올려 “명확한 3권분립으로 정치의 사법 개입이 금지된 대한민국은 정치의 사법 판결 개입은 불법이고 상식적으로 있을 수 없는 일이기 때문에 일본의 ‘징용판결에 대한 정치개입’ 요구를 이해할 수도, 수용할 수도 없다”고 지적했다. 이어 “법적으로나 국민감정으로나 수용 불가능한 조건을 내세우는 것을 보니 스가총리가 방한할 일은 없을 것 같다”고 주장했다. ‘한국이 피고인 일본 기업 자산을 매각하지 않는다고 약속해야 스가 총리가 방한할 수 있다’는 한 일본 관리 발언에 대한 반응으로 보이는 이 지사의 이같은 언급은 이 대표와는 대비된다. 이 대표는 스가 총리 취임을 계기로 “일본의 국운이 상승하고 한·일관계가 개선되길 바란다”며 기대감을 밝힌 바 있다. 이 지사는 “일본과 한국은 복잡하고 미묘한 역사적, 국제정치학적, 외교군사적, 경제사회적 문제들을 해결하기 위해 정치외교와 경제사회 분리, 상호존중과 이해라는 큰 원칙을 지켜야 한다”며 “일본이 아무리 부인해도 침략과 잔혹한 인권침해의 역사는 대한민국에게 역사적 진실이자, 현실”이라고 못박았다. 그는 “위안부, 강제노역 문제는 누가 뭐라하든 가해자인 일본이 만든 문제”라며 “진정한 화해를 위한 사과는 피해자가 용서하고 그만하라 할 때까지 진심으로 하는 것이지 ‘옜다, 사과’로 쉽게 끝낼 수 있는 것이 아니다”고 강조했다. 그러면서 “일본과 한국은 복잡하고 미묘한 역사적, 국제정치학적, 외교 군사적, 경제 사회적 문제들을 해결하기 위해 정치 외교와 경제사회 분리, 상호존중과 이해라는 큰 원칙을 지켜야 한다”고 주문했다.')

이재명 '징용판결 '정치개입' 요구한 스가, 방한할 일 없을 것'


'“文정부, 이체하고 묻은 것”...페북, 4개 기관에 ‘디지털 뉴딜’ 요구'

In [28]:
# https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=022&aid=0003508683
print("文 대통령 “유명희 지지 요청”… 메르켈 “전문성 갖춘 적임자”")
summarize('문재인 대통령이 세계무역기구(WTO) 사무총장 선거에 출마한 유명희 산업통상자원부 통상교섭본부장을 돕기 위해 적극 나섰다. 추석인 1일 앙겔라 메르켈 독일 총리와 정상통화를 갖고 유 본부장 지지를 당부했다. 통화는 문 대통령 제의로 이날 오후 6시부터 약 20분간 이어졌다. 양국 정상의 직접 소통은 약 2년만이다. 두 정상은 2018년 10월 아셈(ASEM·아시아유럽정상회의) 정상회의를 계기로 벨기에 브뤼셀에서 정상회담을 한 바 있다. 문 대통령은 지난달 24일엔 유 본부장 지지 요청 서한을 독일 측에 전달한 바 있다. 문 대통령은 이날 통화에서 “한국은 자유무역질서 속에서 성장해왔고 다자무역체제의 수호와 발전이 WTO 중심으로 이뤄져야 한다는 확고한 신념을 갖고 있다”고 설명했다. 그러면서 “유 본부장은 이런 신념을 실현할 수 있는 비전과 역량을 갖추고 있고 WTO를 발전시키고 신뢰를 회복시킬 수 있는 최적임자라고 생각한다”고 강조했다. 메르켈 총리는 “한국의 유명희 후보가 능력과 전문성을 갖춘 적임자로 보고 있다”고 화답했다고 청와대는 전했다. 문 대통령은 또 오는 3일 독일이 통일 30주년을 맞는 것에 대해 “한반도 평화와 통일을 희망하는 우리 국민들에게도 많은 영감을 주는 의미있는 날”이라며 축하의 뜻을 전했다. 이어 “코로나19(신종 코로나바이러스 감염증) 상황이 전세계적으로 다시 악화하면서 우려가 클 것”이라면서 “그동안 (메르켈) 총리 리더십 하에 독일이 코로나19 대응에 모범이 돼온 것에 경의를 표한다”고 밝혔다. “앞으로도 인류가 코로나 위기를 극복하는데 선도적인 역할을 해 줄 것으로 기대한다”는 덕담도 곁들였다. 메르켈 총리는 “한국이 통일에 대해 꾸는 꿈을 잘 알고 있다”며 “성대하게 독일통일 30주년 행사를 치르고 싶었으나 코로나19 때문에 그러지 못해 유감”이라고 말했다. 또 “코로나19 확산을 막아온 한국의 대처 방식에 큰 관심이 있다”고 전했다.')

文 대통령 “유명희 지지 요청”… 메르켈 “전문성 갖춘 적임자”


"EU, 2020년 '코로나19' 대응 전략 수립...美·中 갈등 확산"

In [29]:
# https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0010909780
print("‘코로나 블루’ 저소득층에 직격탄")
summarize("[앵커] 코로나19의 영향을 받지 않는 사람은 없지만 모두에게 똑같은 무게로 다가오는 것은 아닙니다. 소득이 낮을수록 코로나 이후 소득이 더 줄었고, 우울감도 더 많이 느끼는 것으로 나타났는데요, KBS의 추석민심 여론 조사, 오늘(1일)은 코로나19로 우리 삶이 얼마나 달라졌는지 손은혜 기자가 전해드립니다. [리포트] 먼저 정부의 코로나19 방역 대응에 대해 물었습니다. '잘하고 있다'는 응답이 79.9%, '잘못하고 있다'는 응답이 18.8%로, 긍정 평가가 부정 평가보다 4배 넘게 높았습니다. 전국민에게 지원금을 지급했던 1차 때와 달리, 선별적으로 지원한 이번 재난지원금에 대해서는 전체의 60.9%가 잘했다고 응답했습니다. 코로나 여파로 불안감이나 우울감을 느끼는 이른바 '코로나 블루'는 수치로 확인됐습니다. 태풍이나 지진 등 자연재해에 대해 57.7%가 불안감을 느낀다고 했는데 독감이나 코로나19 등 감염병에 대해선 81%가 불안감을 느낀다고 답했습니다. 코로나 이후 우울해진 경우가 많아졌냐는 질문에는 절반 이상인 54.7%가 '많아졌다'고 답했습니다. 특히 가구소득 월 100만원 미만에서 평균보다 10% 포인트 정도 높은 65.6%가 우울감이 많아졌다고 답했습니다. 소득이 낮을수록 재난으로 인한 우울감을 더 크게 느끼고 있는 겁니다. 연령이 높아질수록 우울감이 증가하는 경향도 나타났는데, 2,30대에선 우울감이 많아졌다는 응답이 절반 이하인 반면 40대 이상부터는 그 응답이 더 많아졌습니다. 실제 소득에는 어떤 변화가 있었는지 물었더니 전체의 절반 이상인 50.4%가 소득이 줄었다고 답했습니다. 역시 저소득층이 소득 감소를 더 크게 체감했는데, 월소득 100만원 미만의 경우 전체의 63%가 소득이 감소했다고 응답했습니다. 취약계층일수록 코로나 이후 충격파가 더 컸다는 사실이 수치로 확인된 셈인데, 앞으로의 재난대응 방향에 시사점을 던져주고 있습니다.")

‘코로나 블루’ 저소득층에 직격탄


"코로나19에 지친 시민, 'LG화학'에 대한 '팽팽'"

# 101(경제)

In [30]:
print("손병석 한국철도 사장, 고양 KTX차량기지 안전 점검")
summarize('[파이낸셜뉴스] 손병석 한국철도(코레일) 사장이 1일 오후 경기도 고양에 있는 수도권철도차량정비단을 찾아 KTX 정비상황을 점검했다. 손병석 사장은 KTX 열차 운행 전 유지관리 상황을 살피고 만일의 상황을 대비한 비상대응체계를 점검했다. 손병석 사장은 “추석 대수송기간 KTX 열차 운행이 늘어난 만큼 더욱 꼼꼼한 정비가 필요하다”며 “특히 방역과 차내설비 점검에 만전을 기해 국민들이 안심하고 편안히 고향을 다녀올 수 있도록 최선을 다해달라”고 강조하고 직원들을 격려했다. 한편 한국철도는 4일까지를 추석 특별교통대책기간으로 정하고, 특별교통대책본부를 24시간 운영하고 있다.')

손병석 한국철도 사장, 고양 KTX차량기지 안전 점검


"'디지털 교도소' 운영 중단"

In [31]:
print("금리인하 비대면 신청 급증…3년간 아낀 이자 '1100억'")
summarize("[아시아경제 조강욱 기자] 신용이 개선된 대출 고객들이 은행에 금리 인하를 요구해 3년 반 동안 아낀 돈이 1130억원을 넘어선 것으로 나타났다. 1일 국회 정무위원회 간사인 더불어민주당 김병욱 의원이 금융감독원으로부터 제출받은 자료에 따르면 KB국민ㆍ신한ㆍ우리ㆍ하나ㆍSC제일ㆍ씨티ㆍ기업은행과 케이ㆍ카카오뱅크에 올해 상반기 접수된 금리 인하 요구는 33만8082건으로 집계됐다. 2017년 11만371건에서 2018년 22만8558건, 2019년 47만8150건으로 늘어난 데 이어 증가세가 더 가팔라진 추세다. 금리인하요구권은 신용평가 등급이 올랐거나 취업ㆍ승진을 했을 때, 재산이 늘었을 때 개선된 신용 상태를 반영해 대출 이자를 깎아달라고 요구할 수 있는 권리로 지난해 6월 법제화를 계기로 활성화됐다. 특히 전체 금리 인하 요구 중 비대면 신청의 비중은 2017년 60.3%에서 2018년 85.9%, 2019년 95.2%, 2020년 상반기 98.2%로 급증했다. 대부분이 은행 지점 방문 없이 손쉽게 온라인으로 대출 이자를 아끼고 있는 것이다. 금리인하요구권 수용 건수는 2017년 4만5820건에서 2018년 6만877건, 2019년 14만3059건, 올 상반기 14만3059건으로 늘었다. 또 수용에 따른 이자 절감 추정액은 같은 기간 438억800만원, 327억9200만원, 277억3100만원, 93억2200만원 등으로 매년 감소했다. 금리인하요구권 수용률은 2017년 41.5%, 2018년 26.6%, 2019년 29.9%, 올해 상반기 32.5% 수준이었다. 김병욱 의원은 '더 많은 국민이 자신의 권리를 누리고 혜택을 받을 수 있도록 은행권과 금융당국이 적극적으로 금리인하요구권을 홍보하고 수용률 제고를 위해 노력해야 한다'고 강조했다.")

금리인하 비대면 신청 급증…3년간 아낀 이자 '1100억'


'"신생아 투자하려면도 안 돼"...코로나19에 태풍까지 피해 속출'

In [33]:
# https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=421&aid=0004903011
print("중국 '우리 백신 비쌀 거다? 합리적 가격에 판매할 것'")
summarize('(서울=뉴스1) 권영미 기자 = 중국 정부가 코로나19 백신을 개발한다면 합리적 가격에 전세계에 제공할 것이라고 밝혔다. 최근 나온 중국이 개발하는 백신이 서방보다 더 비싸게 가격이 매겨질 것이라는 일부 보도를 일축한 것이다. 1일 차이나데일리에 따르면 왕원빈 중국 외교부 대변인은 이날 뉴스 브리핑에서 "다른 나라에서 개발중인 백신의 가격이 얼마인지는 아직 알 수 없다"면서도 "하지만 중국이 개발하는 백신은 합리적인 가격에 세계에 판매할 것임은 확실히 말할 수 있다"고 설명했다. 이어 "중국은 기부나 보조금 지원 등 다양한 방법으로 개발도상국에 백신을 우선 공급할 것"이라고 덧붙였다. 왕 대변인은 "중국은 분명한 약속을 했고, 우리는 항상 우리의 약속을 존중한다"고 말했다. 시진핑 중국 국가주석도 지난 5월 제73차 세계보건총회(WHA)에서 중국의 코로나백신이 출시되면 글로벌 공공재가 되도록 할 것이며 개발도상국도 싸게 살 수 있게 하겠다고 밝힌 바 있다.')

중국 '우리 백신 비쌀 거다? 합리적 가격에 판매할 것'


"中, 코로나19 대응 전략 스마트폰 '베스트 정보' 수상"

# 105(IT)

In [34]:
print("마이크로소프트, 신형 노트북 '서피스 랩탑 고' 출시 발표")
summarize("(서울=연합뉴스) 이효석 기자 = 마이크로소프트(MS)는 신형 노트북 '서피스 랩탑 고'(Suface Laptop Go)를 출시한다고 1일 발표했다. MS는 '신종 코로나바이러스 감염증(코로나19)으로 인한 재택근무와 비대면 학습 환경이 장기화하고 있다'면서 '뉴노멀 시대 연결성을 지원하기 위해 서피스 제품의 디자인·색상·가격대를 다양화한다'고 밝혔다. 국내 출시 일정과 가격은 정해지지 않았다. 서피스 랩탑 고는 그동안 출시된 서피스 랩탑 중 가장 가볍고 합리적인 가격으로 나온다고 MS는 설명했다. 12.4인치 픽셀 센스 터치스크린 디스플레이가 장착됐고, 대형 트랙패드와 1.3㎜ 키보드가 정확한 타이핑을 돕는다. 16GB RAM과 256GB 스토리지, 인텔 10세대 i5 쿼드코어 프로세서가 탑재됐다. 구성은 성능과 배터리 수명에 최적화됐다. 배터리 사용시간은 최대 13시간이다. 색상은 아이스 블루, 샌드스톤, 플래티넘 등 3가지로 출시된다. 720p HD 카메라와 스튜디오 마이크는 원격 근무 또는 학습을 지원한다. 옴니소닉 스피커와 돌비 오디오는 영상 통화나 영화·음악 감상 경험을 배가한다. 서피스 랩탑 고는 원터치 지문 인식 로그인 기능으로 안전한 로그인도 지원한다. 원터치 로그인 기능은 원드라이브(OneDrive)의 개인 파일 접근에도 연동된다. MS는 '마이크로소프트365와 온라인 저장 공간 등 클라우드 연결 경험을 극대화했다'면서 '서피스 제품을 처음 쓰는 초보자도 빠르게 적용할 수 있을 것'이라고 덧붙였다. 서피스 프로 X [마이크로소프트 제공. 재판매 및 DB 금지] 이날 MS는 지난해 10월 출시한 태블릿 PC '서피스 프로 X'(Surface Pro X)의 업데이트 버전도 출시한다고 발표했다. 서피스 프로 X 업데이트 버전은 차세대 프로세서가 탑재돼 성능이 빨라지는 게 가장 큰 특징이다. MS SQ1 프로세서의 향상된 버전인 SQ2 프로세서가 탑재돼 현존하는 ARM 기반의 PC 중 가장 빠른 성능을 선보인다고 회사 측은 설명했다. 전력 및 성능 향상으로 배터리 사용시간은 최대 15시간으로 길어졌다. 아이 컨택트(Eye Contact) 기능은 화상 통화를 할 때 서로 눈을 맞추면서 통화하는 느낌이 들도록 시선을 조정해줘서 커뮤니케이션을 돕는다. 시그니처 키보드는 플래티넘, 아이스 블루, 파피레드 등 세 가지 색상이 새로 출시된다. 서피스 프로 X 업데이트 버전의 국내 출시 일정과 가격은 조만간 발표된다. MS는 디자이너 콤팩트 키보드, 무선 블루투스 숫자 패드, 4K 무선 디스플레이 어댑터, 블루투스 인체공학 마우스 등 서피스 제품군을 위한 새로운 액세서리도 출시한다고 덧붙였다.")

마이크로소프트, 신형 노트북 '서피스 랩탑 고' 출시 발표


"MS, '로드 오브 히어로즈' 글로벌 출시"

In [35]:
print("상온 노출 의심 독감백신 접종 전국 231곳서 2000명 육박")
summarize("'상온노출' 의심 인플루엔자(독감)백신 접종자가 연일 급증하고 있다. 당초 보건당국은 '문제의 백신 물량을 맞은 사람이 없다'고 발표했으나 접종자수가 어제 하루에만 548명이 늘어나 2000명에 육박하고 있는 것이다. 또한 예방접종치짐을 위반한 접종사례도 1479명분에 이른다. 이로인해 보건당국의 발표가 무색할 정도로 접종자가 늘어나면서 예방접종사업 전반에 걸쳐 부실관리에 대한 지적이 나오고 있다. 한편 질병청은 접종자 가운데 '이상 반응'을 신고한 사람이 이날 4명 더 늘어 총 8명이라고 밝혔다. 이상 반응을 새로 신고한 4명 가운데 2명은 오한·두통·메스꺼움 등, 1명은 두드러기, 1명은 설사 증상이 있다고 각각 보고했다. 질병청은 이와 관련해 '접종 이후 증상이 있었으나 호전된 상태'라고 설명했다. 앞서 이상 반응이 있다고 보고된 4명 역시 발열, 오한 등의 증상이 있었으나 호전됐다고 질병청은 전했다. 질병관리청은 1일 '인플루엔자 예방접종사업 관련' 참고자료를 내고 '현재 상온 노출 여부를 조사 중인 정부조달 (백신) 물량을 접종한 건수는 어제(9월 30일) 기준으로 총 1910건(명)'이라고 밝혔다. 질병청이 전날 발표한 1362명에 비해 하루 새 548명 늘어난 것이다. 접종이 이뤄진 날짜별로 보면 9월 21일까지 접종받은 사람이 1261명으로 가장 많았다. 이후 22일 431명, 23일 23명, 24일 22명, 25일 96명, 26일 38명, 27일 18명, 28일 21명 등이다. 질병청이 긴급하게 사용 중단 결정을 내리면서 일선 현장에서 혼선이 빚어질 수 있었다고 판단한 22일 당일을 제외한 전후의 접종 사례 1479명분의 물량은 모두 예방접종 지침을 위반한 것이다. 실제 한 의료기관에서는 돈을 내고 접종을 받은 60명이 정부의 무료 물량으로 무더기로 접종받은 사실이 드러났다. 질병청 관계자는 '(무료 접종) 사업 시작 전(22일 이전)과 중단 고지일 이후(23일 이후)에 접종이 이뤄진 사례는 국가 인플루엔자 예방접종 사업 지침을 미준수한 사례'라고 지적했다. 이어 '사업 중단 당일인 22일에 이뤄진 접종 사례는 사업 중단을 인지하지 못하고 접종한 것으로 보고 있다'며 '각 지방자치단체를 통해 사용 중지된 물량을 사용한 사례를 지속해서 조사하고 있다'고 설명했다. 접종자가 나온 지역은 강원과 울산을 제외한 전국 15개 시도다. 지역별로는 경기 673건, 전북 326건, 인천 214건, 경북 161건, 서울 149건, 부산 109건, 충남 74건, 세종 51건, 대구 46건, 광주 40건, 전남 31건, 대전 17건, 경남 10건, 제주 8건, 충북 1건 등이다. ◇ 전국 231곳 의료기관서 접종…이상반응 신고 4명 늘어 총 8명 '국가 인플루엔자 예방접종 사업 중단'이라는 초유의 사태 속에서 그간 일선 의료현장에서 백신 접종 및 관리를 부실하게 해 온 점도 조사 과정에서 속속 드러나고 있다. 통상 각 의료기관이 자체적으로 구비한 유료 접종 물량과 정부가 제공하는 무료 접종 물량은 별도로 관리해야 하지만 이를 섞어서 관리하거나 돈을 받고 정부 조달 물량을 쓰는 경우도 적지 않은 것으로 보인다. 현재까지 상온 노출이 의심돼 조사 중인 백신으로 접종한 병·의원만 하더라도 전국 231곳에 달한다. 지역별로 보면 경기가 93곳으로 가장 많고 이어 전북 31곳, 대구 22곳, 서울 18곳, 경북 15곳 부산·충남 11곳 등이다. 질병청은 앞서 국가 조달 물량을 공급하는 업체인 '신성약품'이 백신을 배송하는 과정에서 냉장차 문을 열어놓거나 제품을 바닥에 내려놓는 등 '냉장유통'(콜드체인) 원칙을 지키지 않은 사실을 확인하고 지난 21일 밤 사업 중단 방침을 전격 발표했다. 상온 노출이 의심돼 사용이 중단된 백신 물량은 총 578만명분이다. 당초 질병청은 백신 사용 중단을 발표한 직후인 지난달 22일 문제의 백신 접종자가 1명도 없다고 밝혔으나 9월 25일 이후부터 105명→224명→324명→407명→873명→1362명→1910명 등으로 연일 불어나고 있다.")

상온 노출 의심 독감백신 접종 전국 231곳서 2000명 육박


'[속보] "코로나19로 인한 소비...수도권서 2명 추가 확진"'

In [44]:
reference = "상온 노출 의심 독감백신 접종 전국 231곳서 2000명 육박"
print("상온 노출 의심 독감백신 접종 전국 231곳서 2000명 육박")

candidate = summarize("'상온노출' 의심 인플루엔자(독감)백신 접종자가 연일 급증하고 있다. 당초 보건당국은 '문제의 백신 물량을 맞은 사람이 없다'고 발표했으나 접종자수가 어제 하루에만 548명이 늘어나 2000명에 육박하고 있는 것이다. 또한 예방접종치짐을 위반한 접종사례도 1479명분에 이른다. 이로인해 보건당국의 발표가 무색할 정도로 접종자가 늘어나면서 예방접종사업 전반에 걸쳐 부실관리에 대한 지적이 나오고 있다. 한편 질병청은 접종자 가운데 '이상 반응'을 신고한 사람이 이날 4명 더 늘어 총 8명이라고 밝혔다. 이상 반응을 새로 신고한 4명 가운데 2명은 오한·두통·메스꺼움 등, 1명은 두드러기, 1명은 설사 증상이 있다고 각각 보고했다. 질병청은 이와 관련해 '접종 이후 증상이 있었으나 호전된 상태'라고 설명했다. 앞서 이상 반응이 있다고 보고된 4명 역시 발열, 오한 등의 증상이 있었으나 호전됐다고 질병청은 전했다. 질병관리청은 1일 '인플루엔자 예방접종사업 관련' 참고자료를 내고 '현재 상온 노출 여부를 조사 중인 정부조달 (백신) 물량을 접종한 건수는 어제(9월 30일) 기준으로 총 1910건(명)'이라고 밝혔다. 질병청이 전날 발표한 1362명에 비해 하루 새 548명 늘어난 것이다. 접종이 이뤄진 날짜별로 보면 9월 21일까지 접종받은 사람이 1261명으로 가장 많았다. 이후 22일 431명, 23일 23명, 24일 22명, 25일 96명, 26일 38명, 27일 18명, 28일 21명 등이다. 질병청이 긴급하게 사용 중단 결정을 내리면서 일선 현장에서 혼선이 빚어질 수 있었다고 판단한 22일 당일을 제외한 전후의 접종 사례 1479명분의 물량은 모두 예방접종 지침을 위반한 것이다. 실제 한 의료기관에서는 돈을 내고 접종을 받은 60명이 정부의 무료 물량으로 무더기로 접종받은 사실이 드러났다. 질병청 관계자는 '(무료 접종) 사업 시작 전(22일 이전)과 중단 고지일 이후(23일 이후)에 접종이 이뤄진 사례는 국가 인플루엔자 예방접종 사업 지침을 미준수한 사례'라고 지적했다. 이어 '사업 중단 당일인 22일에 이뤄진 접종 사례는 사업 중단을 인지하지 못하고 접종한 것으로 보고 있다'며 '각 지방자치단체를 통해 사용 중지된 물량을 사용한 사례를 지속해서 조사하고 있다'고 설명했다. 접종자가 나온 지역은 강원과 울산을 제외한 전국 15개 시도다. 지역별로는 경기 673건, 전북 326건, 인천 214건, 경북 161건, 서울 149건, 부산 109건, 충남 74건, 세종 51건, 대구 46건, 광주 40건, 전남 31건, 대전 17건, 경남 10건, 제주 8건, 충북 1건 등이다. ◇ 전국 231곳 의료기관서 접종…이상반응 신고 4명 늘어 총 8명 '국가 인플루엔자 예방접종 사업 중단'이라는 초유의 사태 속에서 그간 일선 의료현장에서 백신 접종 및 관리를 부실하게 해 온 점도 조사 과정에서 속속 드러나고 있다. 통상 각 의료기관이 자체적으로 구비한 유료 접종 물량과 정부가 제공하는 무료 접종 물량은 별도로 관리해야 하지만 이를 섞어서 관리하거나 돈을 받고 정부 조달 물량을 쓰는 경우도 적지 않은 것으로 보인다. 현재까지 상온 노출이 의심돼 조사 중인 백신으로 접종한 병·의원만 하더라도 전국 231곳에 달한다. 지역별로 보면 경기가 93곳으로 가장 많고 이어 전북 31곳, 대구 22곳, 서울 18곳, 경북 15곳 부산·충남 11곳 등이다. 질병청은 앞서 국가 조달 물량을 공급하는 업체인 '신성약품'이 백신을 배송하는 과정에서 냉장차 문을 열어놓거나 제품을 바닥에 내려놓는 등 '냉장유통'(콜드체인) 원칙을 지키지 않은 사실을 확인하고 지난 21일 밤 사업 중단 방침을 전격 발표했다. 상온 노출이 의심돼 사용이 중단된 백신 물량은 총 578만명분이다. 당초 질병청은 백신 사용 중단을 발표한 직후인 지난달 22일 문제의 백신 접종자가 1명도 없다고 밝혔으나 9월 25일 이후부터 105명→224명→324명→407명→873명→1362명→1910명 등으로 연일 불어나고 있다.")

score(reference, candidate)

상온 노출 의심 독감백신 접종 전국 231곳서 2000명 육박


8.726094729337945e-232

In [36]:
print("코로나 가짜뉴스 제1주범은 '살균제 주사해 치료하라’던 이 사람")
summarize('도널드 트럼프 미 대통령이 코로나 관련 가짜뉴스가 퍼지는 데 가장 큰 원인을 제공했다는 연구 결과가 발표됐다. 트럼프 대통령이 인터뷰나 SNS를 통해 코로나에 대한 확인되지 않은 언급을 자주 해 인포데믹(infodemic·잘못된 정보가 빠르게 확산되는 현상)을 일으킨 ‘제1 주범’이라는 것이다. 미 코넬대 연구팀은 30일(현지 시각) “전 세계 영어 매체에 실린 3800만개의 코로나 관련 기사를 분석한 결과 코로나와 관련해 잘못된 정보를 담은 기사의 38%가 트럼프 대통령의 언급을 보도한 것”이라고 밝혔다. 연구진은 “미국 대통령이 코로나 관련 인포데믹의 가장 큰 원인 제공자”라며 “전염병과 관련된 잘못된 정보 확산은 단순 오보에 그치지 않고 사람들의 건강까지 해칠 수 있어 문제”라고 지적했다. 연구진에 따르면 코로나가 본격 유행한 올해 초부터 지난 5월까지 영문 매체에 보도된 코로나 관련 가짜 뉴스는 110만건 정도다. 이들 가짜뉴스는 총 11종류의 코로나 관련 잘못된 정보를 담고 있다고 연구진은 설명했다. 예를 들어 지난 1월 대통령 탄핵 추진 당시 “민주당이 트럼프의 재선을 막기위해 코로나를 퍼뜨렸다”, 글로벌 제약사가 치료제를 팔기 위해 바이러스를 개발했다는 음모론 등이다. “코로나 잘못된 정보의 38%가 트럼프 언급” 연구진은 “이중에서 트럼프 대통령이 지난 4월 한 ‘기적의 치료제’ 언급이 다른 10개 잘못된 코로나 정보보다 많이 보도됐다”고 설명했다. 트럼프 대통령은 지난 4월 “코로나 치료를 위해 살균제를 (사람에) 주사기로 주입하라”고 발언했다. 가디언 등 외신은 “트럼프 대통령의 이 발언 이후 온라인 쇼핑몰 아마존에서는 표백제가 기적의 치료제라는 이름으로 판매됐다”고 보도했다. 트럼프 대통령의 근거 없는 정보가 일반인들의 잘못된 행동을 유발했다는 것이다. 코로나 관련 가짜뉴스 확산에는 트럼프 대통령의 SNS도 한몫을 했다. 트럼프 대통령은 최근 자신의 페이스북, 유튜브 계정에 올린 언론 인터뷰 영상에서 “아이들은 대부분 코로나 등 질병에 면역력이 있다. 우리(성인)보다 훨씬 면역력이 강해 문제가 없다”고 했다. 이 게시물은 페이스북과 유튜브에서 모두 삭제 조치 당했다. 코로나 관련 잘못된 정보를 담고 있다는 이유 때문이다. 페이스북 측은 당시 “(트럼프 대통령의) 영상은 특정 집단이 코로나에 면역력을 갖췄다는 잘못된 정보를 포함하고 있으며 이는 코로나 방역에 대한 해로운 거짓 정보”라고 밝혔다. 연구진은 “코로나와 같은 전염병 관련 가짜뉴스는 미국이 다른 나라들처럼 전염병 확산에 제대로 대처하지 못하게 한 원인의 하나”라며 “치료제나 백신이 없는 상황에서는 바이러스에 대한 정확한 정보가 중요하다”고 밝혔다.')

코로나 가짜뉴스 제1주범은 '살균제 주사해 치료하라’던 이 사람


'트럼프, ‘코로나19’에 · 국민의힘·페북도 \'발끈\'..."난 유전자보다 더 중요"'

In [43]:
reference = "코로나 가짜뉴스 제1주범은 '살균제 주사해 치료하라’던 이 사람"
candidate = summarize('도널드 트럼프 미 대통령이 코로나 관련 가짜뉴스가 퍼지는 데 가장 큰 원인을 제공했다는 연구 결과가 발표됐다. 트럼프 대통령이 인터뷰나 SNS를 통해 코로나에 대한 확인되지 않은 언급을 자주 해 인포데믹(infodemic·잘못된 정보가 빠르게 확산되는 현상)을 일으킨 ‘제1 주범’이라는 것이다. 미 코넬대 연구팀은 30일(현지 시각) “전 세계 영어 매체에 실린 3800만개의 코로나 관련 기사를 분석한 결과 코로나와 관련해 잘못된 정보를 담은 기사의 38%가 트럼프 대통령의 언급을 보도한 것”이라고 밝혔다. 연구진은 “미국 대통령이 코로나 관련 인포데믹의 가장 큰 원인 제공자”라며 “전염병과 관련된 잘못된 정보 확산은 단순 오보에 그치지 않고 사람들의 건강까지 해칠 수 있어 문제”라고 지적했다. 연구진에 따르면 코로나가 본격 유행한 올해 초부터 지난 5월까지 영문 매체에 보도된 코로나 관련 가짜 뉴스는 110만건 정도다. 이들 가짜뉴스는 총 11종류의 코로나 관련 잘못된 정보를 담고 있다고 연구진은 설명했다. 예를 들어 지난 1월 대통령 탄핵 추진 당시 “민주당이 트럼프의 재선을 막기위해 코로나를 퍼뜨렸다”, 글로벌 제약사가 치료제를 팔기 위해 바이러스를 개발했다는 음모론 등이다. “코로나 잘못된 정보의 38%가 트럼프 언급” 연구진은 “이중에서 트럼프 대통령이 지난 4월 한 ‘기적의 치료제’ 언급이 다른 10개 잘못된 코로나 정보보다 많이 보도됐다”고 설명했다. 트럼프 대통령은 지난 4월 “코로나 치료를 위해 살균제를 (사람에) 주사기로 주입하라”고 발언했다. 가디언 등 외신은 “트럼프 대통령의 이 발언 이후 온라인 쇼핑몰 아마존에서는 표백제가 기적의 치료제라는 이름으로 판매됐다”고 보도했다. 트럼프 대통령의 근거 없는 정보가 일반인들의 잘못된 행동을 유발했다는 것이다. 코로나 관련 가짜뉴스 확산에는 트럼프 대통령의 SNS도 한몫을 했다. 트럼프 대통령은 최근 자신의 페이스북, 유튜브 계정에 올린 언론 인터뷰 영상에서 “아이들은 대부분 코로나 등 질병에 면역력이 있다. 우리(성인)보다 훨씬 면역력이 강해 문제가 없다”고 했다. 이 게시물은 페이스북과 유튜브에서 모두 삭제 조치 당했다. 코로나 관련 잘못된 정보를 담고 있다는 이유 때문이다. 페이스북 측은 당시 “(트럼프 대통령의) 영상은 특정 집단이 코로나에 면역력을 갖췄다는 잘못된 정보를 포함하고 있으며 이는 코로나 방역에 대한 해로운 거짓 정보”라고 밝혔다. 연구진은 “코로나와 같은 전염병 관련 가짜뉴스는 미국이 다른 나라들처럼 전염병 확산에 제대로 대처하지 못하게 한 원인의 하나”라며 “치료제나 백신이 없는 상황에서는 바이러스에 대한 정확한 정보가 중요하다”고 밝혔다.')
score(reference, candidate)

9.788429383461836e-232

In [37]:
from nltk.translate.bleu_score import sentence_bleu

reference = summary_tokenizer.EncodeAsPieces("상온 노출 의심 독감백신 접종 전국 231곳서 2000명 육박")
candidate = summary_tokenizer.EncodeAsPieces('[속보] "코로나19로 인한 소비...수도권서 2명 추가 확진"')

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu(reference, candidate))

원문: ['<BOS>', '▁상온', '▁노출', '▁의심', '▁독감백신', '▁접종', '▁전국', '▁231', '곳서', '▁2000', '명', '▁육박', '<EOS>']
번역문: ['<BOS>', '▁[', '속보', ']', '▁"', '코로나', '19', '로', '▁인', '한', '▁소비', '...', '수도권서', '▁2', '명', '▁추가', '▁확진', '"', '<EOS>']
BLEU Score: 8.726094729337945e-232


/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contain

In [42]:
def score(reference, candidate):
    reference = summary_tokenizer.EncodeAsPieces(reference)
    candidate = summary_tokenizer.EncodeAsPieces(candidate)
    return sentence_bleu(reference, candidate)